In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/My Drive/Data.zip"

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run lllor pressing Shift+Enter) will list the files in the input directory

import os


# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir("Data"))

In [ ]:
DATA_PATH = "Data/train_data/"
TEST_DATA_PATH ="Data/test_data/"

In [ ]:
PRETRAINED_MODEL = None

In [ ]:
import h5py
import torch
import shutil

def save_net(fname, net):
    with h5py.File(fname, 'w') as h5f:
        for k, v in net.state_dict().items():
            h5f.create_dataset(k, data=v.cpu().numpy())
def load_net(fname, net):
    with h5py.File(fname, 'r') as h5f:
        for k, v in net.state_dict().items():        
            param = torch.from_numpy(np.asarray(h5f[k]))         
            v.copy_(param)
            
def save_checkpoint(state, is_best,task_id, filename='checkpoint.pth.tar'):
    torch.save(state, task_id+filename)
    if is_best:
        shutil.copyfile(task_id+filename, task_id+'model_best.pth.tar')   

In [ ]:
"""
contain dummy args with config
helpfull for copy paste Kaggle
"""
import argparse


def make_args(train_json= "", test_json="", pre="", gpu="0", task="task_one_"):
    """
    these arg does not have any required commandline arg (all with default value)
    :param train_json:
    :param test_json:
    :param pre:
    :param gpu:
    :param task:
    :return:
    """
    parser = argparse.ArgumentParser(description='PyTorch CSRNet')

    args = parser.parse_args()
    args.gpu = gpu
    args.task = task
    args.pre = None
    return args


class Meow():
    def __init__(self):
        pass


def make_meow_args(gpu="0", task="task_one_"):
    args = Meow()
    args.gpu = gpu
    args.task = task
    args.pre = None
    return args

In [ ]:
import os
import glob
from sklearn.model_selection import train_test_split
import json
"""
create a list of file (full directory)
"""

def create_training_image_list(data_path):
    """
    create a list of absolutely path of jpg file
    :param data_path: must contain subfolder "images" with *.jpg  (example ShanghaiTech/part_A/train_data/)
    :return:
    """
    DATA_PATH = data_path
    image_path_list = glob.glob(os.path.join(DATA_PATH, "images", "*.jpg"))
    return image_path_list

def get_train_val_list(data_path):
    DATA_PATH = data_path
    image_path_list = glob.glob(os.path.join(DATA_PATH, "images", "*.jpg"))
    train, val = train_test_split(image_path_list, test_size=0.3, random_state=113)

    print("train size ", len(train))
    print("val size ", len(val))
    return train, val

In [ ]:
import random
import os
from PIL import Image,ImageFilter,ImageDraw
import numpy as np
import h5py
from PIL import ImageStat
import cv2

def load_data(img_path,train = True):
    gt_path = img_path.replace('.jpg','.h5').replace('images','ground-truth')
    img = Image.open(img_path).convert('RGB')
    gt_file = h5py.File(gt_path, 'r')
    target = np.asarray(gt_file['density'])

    target = cv2.resize(target,(int(target.shape[1]/8), int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64
    
    return img,target

In [ ]:
import os
import random
import torch
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms.functional as F
class ListDataset(Dataset):
    def __init__(self, root, shape=None, shuffle=True, transform=None,  train=False, seen=0, batch_size=1, num_workers=4):
        """
        if you have different image size, then batch_size must be 1
        :param root:
        :param shape:
        :param shuffle:
        :param transform:
        :param train:
        :param seen:
        :param batch_size:
        :param num_workers:
        """
        if train:
            root = root *2
        if shuffle:
            random.shuffle(root)
        
        self.nSamples = len(root)
        self.lines = root
        self.transform = transform
        self.train = train
        self.shape = shape
        self.seen = seen
        self.batch_size = batch_size
        self.num_workers = num_workers
        
    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        assert index <= len(self), 'index range error' 
        
        img_path = self.lines[index]
        
        img,target = load_data(img_path,self.train)
        
        #img = 255.0 * F.to_tensor(img)
        
        #img[0,:,:]=img[0,:,:]-92.8207477031
        #img[1,:,:]=img[1,:,:]-95.2757037428
        #img[2,:,:]=img[2,:,:]-104.877445883


        
        
        if self.transform is not None:
            img = self.transform(img)
        return img,target

In [ ]:
import os
from easydict import EasyDict as edict
import time
import torch

# init
__C = edict()

cfg = __C
__C.DATA = edict()
__C.TRAIN = edict()
__C.VAL = edict()
__C.VIS = edict()

#------------------------------DATA------------------------
__C.DATA.STD_SIZE = (576,768)
__C.DATA.DATA_PATH = '/media/D/DataSet/CC/' +str(__C.DATA.STD_SIZE[0]) + 'x' + str(__C.DATA.STD_SIZE[1]) + '/shanghaitech_part_B'                  
__C.DATA.MEAN_STD = ([0.444637000561], [0.226200059056]) # part B
__C.DATA.DEN_ENLARGE = 1.

#------------------------------TRAIN------------------------
__C.TRAIN.INPUT_SIZE = (512,680)
__C.TRAIN.SEED = 640
__C.TRAIN.RESUME = ''#model path
__C.TRAIN.BATCH_SIZE = 6 #imgs
__C.TRAIN.BCE_WIEGHT = 1e-4

__C.TRAIN.SEG_LR = 1e-2
__C.TRAIN.SEG_WIEGHT = 1e-4

__C.TRAIN.NUM_BOX = 20 #boxes

__C.TRAIN.GPU_ID = [1]

# base lr
__C.TRAIN.LR = 1e-4
__C.TRAIN.LR_DECAY = 1
__C.TRAIN.NUM_EPOCH_LR_DECAY = 1 # epoches

__C.TRAIN.MAX_EPOCH = 2000

# output 
__C.TRAIN.PRINT_FREQ = 10

now = time.strftime("%m-%d_%H-%M", time.localtime())

__C.TRAIN.EXP_NAME = 'PCC_Net' + now

__C.TRAIN.EXP_PATH = './exp'

#------------------------------VAL------------------------
__C.VAL.BATCH_SIZE = 1 # imgs
__C.VAL.FREQ = 1

#------------------------------VIS------------------------
__C.VIS.VISIBLE_NUM_IMGS = 20

#------------------------------MISC------------------------


#================================================================================
#================================================================================
#================================================================================  

In [ ]:
from torch import nn
import torch.nn.functional as F
import numpy as np
from PIL import Image
import os
import shutil

import pdb
import math



def weights_normal_init(*models):
    for model in models:
        dev=0.01
        if isinstance(model, list):
            for m in model:
                weights_normal_init(m, dev)
        else:
            for m in model.modules():            
                if isinstance(m, nn.Conv2d):        
                    m.weight.data.normal_(0.0, dev)
                    if m.bias is not None:
                        m.bias.data.fill_(0.0)

                elif isinstance(m, nn.Linear):
                    m.weight.data.normal_(0.0, dev)


def initialize_weights(*models):
    for model in models:
        for module in model.modules():
            if isinstance(module, nn.Conv2d):
                n = module.kernel_size[0] * module.kernel_size[1] * module.out_channels
                module.weight.data.normal_(0, math.sqrt(2. / n))
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()
            elif isinstance(module, nn.Linear):
                n = module.weight.size(1)
                module.weight.data.normal_(0, math.sqrt(2. / n))
                module.bias.data.zero_()


# def initialize_weights(models):
#     for model in models:
#         real_init_weights(model)


# def real_init_weights(m):

#     if isinstance(m, list):
#         for mini_m in m:
#             real_init_weights(mini_m)
#     else:
#         if isinstance(m, nn.Conv2d):    
#             nn.init.normal_(m.weight, std=0.01)
#             if m.bias is not None:
#                 nn.init.constant_(m.bias, 0)
#         elif isinstance(m, nn.Linear):
#             m.weight.data.normal_(0.0, std=0.01)
#         elif isinstance(m, nn.BatchNorm2d):
#             nn.init.constant_(m.weight, 1)
#             nn.init.constant_(m.bias, 0)
#         elif isinstance(m,nn.Module):
#             for mini_m in m.children():
#                 real_init_weights(mini_m)
#         else:
#             print( m )

def weights_init_kaiming(m):
    if isinstance(m, nn.Conv2d):
        #kaiming is first name of author whose last name is 'He' lol
        nn.init.kaiming_uniform(m.weight) 
        m.bias.data.zero_()

def adjust_learning_rate(lr, decay, optimizer, cur_epoch, n_epochs):
    """Sets the learning rate to the initially 
        configured `lr` decayed by `decay` every `n_epochs`"""
    new_lr = lr * (decay ** (cur_epoch // n_epochs))
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr

def calculate_mean_iu(predictions, gts, num_classes):
    sum_iu = 0
    class_iu = []
    for i in xrange(num_classes):
        n_ii = t_i = sum_n_ji = 1e-9
        for p, gt in zip(predictions, gts):
            n_ii += np.sum(gt[p == i] == i)
            t_i += np.sum(gt == i)
            sum_n_ji += np.sum(p == i)
        sum_iu += float(n_ii) / (t_i + sum_n_ji - n_ii)
        class_iu.append(float(n_ii) / (t_i + sum_n_ji - n_ii))
    mean_iu = sum_iu / num_classes
    return mean_iu,class_iu


def calculate_lane_metrics(predictions, gts, num_classes):
    sum_iu = 0
    class_iu = []
    acc = []
    rec = []
    f1_m = []
    for i in xrange(num_classes):
        tp = fp = fn = 0.
        for p, gt in zip(predictions, gts):
            tp += np.sum(gt[p == i] == i)
            fp += np.sum( (gt[p == i] != i ))
            fn += np.sum(gt[p != i] == i)

        class_iu.append(tp / (tp + fp + fn + 1e-9))
        acc.append(tp/(tp+fp+1e-9))
        rec.append(tp/(tp+fn+1e-9))
        f1_m.append(2*acc[i]*rec[i]/(acc[i]+rec[i]+1e-9))
        sum_iu += tp / (tp + fp + fn + 1e-9)
    mean_iu = sum_iu / num_classes
    return {'miu':mean_iu,
            'ciu':class_iu
            },\
            {'acc':acc,
             'rec':rec,
             'f1_m':f1_m
            }


class CrossEntropyLoss2d(nn.Module):
    def __init__(self):
        super(CrossEntropyLoss2d, self).__init__()
        self.nll_loss = nn.NLLLoss(weight=None, reduction='mean')

    def forward(self, inputs, targets):
        # pdb.set_trace()
        return self.nll_loss(F.log_softmax(inputs,dim=1), targets)

def rmrf_mkdir(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.mkdir(dir_name)

def rm_file(path_file):
    if os.path.exists(path_file):
        os.remove(path_file)

def colorize_mask(mask):
    # mask: numpy array of the mask
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(cfg.VIS.PALETTE_LABEL_COLORS)

    return new_mask

def scores(predictions, gts, num_classes):
    hist = np.zeros((num_classes, num_classes))
    for lp, lt in zip(predictions, gts):
        hist += _fast_hist(lp.flatten(), lt.flatten(), num_classes)
    # axis 0: gt, axis 1: prediction
    # acc = np.diag(hist).sum() / hist.sum()
    # acc_cls = np.diag(hist) / hist.sum(axis=1)
    # acc_cls = np.nanmean(acc_cls)
    iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
    mean_iu = np.nanmean(iu)
    # freq = hist.sum(axis=1) / hist.sum()
    # fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    # return acc, acc_cls, mean_iu, fwavacc
    # pdb.set_trace()
    return {'miu':mean_iu,
            'ciu':iu
            }

def acc(predictions, gts, num_classes=2):
    predictions = predictions.data.numpy().astype(np.int64)
    gts = gts.data.numpy().astype(np.int64)
    predictions[predictions>=0.5]=1
    predictions[predictions<0.5]=0

    t = predictions==gts
    acc = np.sum(t)/float((t.shape[0]*t.shape[1]))
    return acc


def _fast_hist(label_pred, label_true, num_classes):
    mask = (label_true >= 0) & (label_true < num_classes)
    hist = np.bincount(
        num_classes * label_true[mask].astype(int) +
        label_pred[mask], minlength=num_classes ** 2).reshape(num_classes, num_classes)
    return hist

def streaming_scores(hist,predictions, gts, num_classes):
    for lp, lt in zip(predictions, gts):
        hist += _fast_hist(lp.flatten(), lt.flatten(), num_classes)
    # axis 0: gt, axis 1: prediction
    # acc = np.diag(hist).sum() / hist.sum()
    # acc_cls = np.diag(hist) / hist.sum(axis=1)
    # acc_cls = np.nanmean(acc_cls)
    iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
    mean_iu = np.nanmean(iu)
    # freq = hist.sum(axis=1) / hist.sum()
    # fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    # return hist, acc, acc_cls, mean_iu, fwavacc
    return hist, \
        {'miu':mean_iu,
        'ciu':iu
        }

In [ ]:
import torch
import torch.nn as nn
import pdb

class convDU(nn.Module):

    def __init__(self,
        in_out_channels=2048,
        kernel_size=(9,1)
        ):
        super(convDU, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_out_channels, in_out_channels, kernel_size, stride=1, padding=((kernel_size[0]-1)//2,(kernel_size[1]-1)//2)),
            nn.PReLU()
            )

    def forward(self, fea):
        n, c, h, w = fea.size()

        fea_stack = []
        for i in range(h):
            i_fea = fea.select(2, i).reshape(n,c,1,w)
            if i == 0:
                fea_stack.append(i_fea)
                continue
            fea_stack.append(self.conv(fea_stack[i-1])+i_fea)
            # pdb.set_trace()
            # fea[:,i,:,:] = self.conv(fea[:,i-1,:,:].expand(n,1,h,w))+fea[:,i,:,:].expand(n,1,h,w)


        for i in range(h):
            pos = h-i-1
            if pos == h-1:
                continue
            fea_stack[pos] = self.conv(fea_stack[pos+1])+fea_stack[pos]
        # pdb.set_trace()
        fea = torch.cat(fea_stack, 2)
        return fea

class convLR(nn.Module):

    def __init__(self,
        in_out_channels=2048,
        kernel_size=(1,9)
        ):
        super(convLR, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_out_channels, in_out_channels, kernel_size, stride=1, padding=((kernel_size[0]-1)//2,(kernel_size[1]-1)//2)),
            nn.PReLU()
            )

    def forward(self, fea):
        n, c, h, w = fea.size()

        fea_stack = []
        for i in range(w):
            i_fea = fea.select(3, i).reshape(n,c,h,1)
            if i == 0:
                fea_stack.append(i_fea)
                continue
            fea_stack.append(self.conv(fea_stack[i-1])+i_fea)

        for i in range(w):
            pos = w-i-1
            if pos == w-1:
                continue
            fea_stack[pos] = self.conv(fea_stack[pos+1])+fea_stack[pos]


        fea = torch.cat(fea_stack, 3)
        return fea


class Conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, NL='relu', same_padding=False, bn=False, dilation=1):
        super(Conv2d, self).__init__()
        padding = int((kernel_size - 1) // 2) if same_padding else 0
        self.conv = []
        if dilation==1:
            self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding, dilation=dilation)
        else:
            self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=dilation, dilation=dilation)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0, affine=True) if bn else None
        if NL == 'relu' :
            self.relu = nn.ReLU(inplace=True) 
        elif NL == 'prelu':
            self.relu = nn.PReLU() 
        else:
            self.relu = None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x


class FC(nn.Module):
    def __init__(self, in_features, out_features, NL='relu'):
        super(FC, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
        if NL == 'relu' :
            self.relu = nn.ReLU(inplace=True) 
        elif NL == 'prelu':
            self.relu = nn.PReLU() 
        else:
            self.relu = None

    def forward(self, x):
        x = self.fc(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


from torchvision.ops import RoIPool
import pdb

def make_layers(cfg, in_channels = 256 ,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


class ori(nn.Module):

    def __init__(self, bn=False, num_classes=10):
        super(ori, self).__init__()
        self.seen = 0        
        self.num_classes = num_classes        
        self.base_layer = nn.Sequential(Conv2d( 3, 16, 9, same_padding=True, NL='prelu', bn=bn),                                     
                                        Conv2d(16, 32, 7, same_padding=True, NL='prelu', bn=bn))
        
        self.hl_prior = nn.Sequential(Conv2d( 32, 16, 9, same_padding=True, NL='prelu', bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(16, 32, 7, same_padding=True, NL='prelu', bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(32, 32, 7, same_padding=True, NL='prelu', bn=bn),
                                     Conv2d(32, 32, 7, same_padding=True, NL='prelu', bn=bn))
                
        self.roi_pool = RoIPool([16, 16], 1/4.0)
        self.hl_prior_conv2d = Conv2d( 32, 16, 1, same_padding=True, NL='prelu', bn=bn)
        
        self.bbx_pred = nn.Sequential(
                                    FC(16*16*16,512, NL='prelu'),
                                    FC(512,256,  NL='prelu'),
                                    FC(256, self.num_classes, NL='prelu')
                                    )
             
        # generate dense map
        self.den_stage_1 = nn.Sequential(Conv2d( 32, 32, 7, same_padding=True, NL='prelu', bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(32, 64, 5, same_padding=True, NL='prelu', bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(64, 32, 5, same_padding=True, NL='prelu', bn=bn),
                                     Conv2d(32, 32, 5, same_padding=True, NL='prelu', bn=bn))
        
        self.den_stage_DULR = nn.Sequential(convDU(in_out_channels=32,kernel_size=(1,9)),
                                        convLR(in_out_channels=32,kernel_size=(9,1)))


        self.den_stage_2 = nn.Sequential(Conv2d( 64, 64, 3, same_padding=True, NL='prelu', bn=bn),
                                        Conv2d( 64, 32, 3, same_padding=True, NL='prelu', bn=bn),                                        
                                        nn.ConvTranspose2d(32,16,4,stride=2,padding=1,output_padding=0,bias=True),
                                        nn.PReLU(),
                                        nn.ConvTranspose2d(16,8,4,stride=2,padding=1,output_padding=0,bias=True),
                                        nn.PReLU())

        # generrate seg map
        self.seg_stage = nn.Sequential(Conv2d( 32, 32, 1, same_padding=True, NL='prelu', bn=bn),
        	                            Conv2d( 32, 64, 3, same_padding=True, NL='prelu', bn=bn),
                                        Conv2d( 64, 32, 3, same_padding=True, NL='prelu', bn=bn),                                        
                                        nn.ConvTranspose2d(32,16,4,stride=2,padding=1,output_padding=0,bias=True),
                                        nn.PReLU(),
                                        nn.ConvTranspose2d(16,8,4,stride=2,padding=1,output_padding=0,bias=True),
                                        nn.PReLU())

        self.seg_pred = Conv2d(8, 2, 1, same_padding=True, NL='relu', bn=bn)

        self.trans_den = Conv2d(8, 8, 1, same_padding=True, NL='relu', bn=bn)

        self.den_pred = Conv2d(16, 1, 1, same_padding=True, NL='relu', bn=bn)

        self.max1=nn.MaxPool2d(kernel_size=2, stride=2)
 

        #self.frontend = make_layers(self.frontend_feat)
        # self.mydme = my_make_layers(self.mydme_feat,in_channels = 64)
    
        #self.output_layer = nn.Conv2d(1, 1, kernel_size=1)
        # initialize_weights(self.modules())

        #weights_normal_init(self.base_layer, self.hl_prior, self.hl_prior_conv2d, self.bbx_pred, self.den_stage_1, \
        #                    self.den_stage_DULR, self.den_stage_2, self.trans_den, self.den_pred)
        #initialize_weights(self.seg_stage,  self.seg_pred)

        
    def forward(self, im_data):
        x_base = self.base_layer(im_data)

        x_hlp = self.hl_prior(x_base)
        #x_bbx = self.roi_pool(x_hlp, [16, 16])
        #x_bbx = self.hl_prior_conv2d(x_bbx)

        #x_bbx = x_bbx.view(x_bbx.size(0), -1) 
        #x_bbx = self.bbx_pred(x_bbx)

        x_map_1 = self.den_stage_1(x_base)
        x_map_1 = self.den_stage_DULR(x_map_1) 

        x_map = torch.cat((x_hlp,x_map_1),1)
        x_map = self.den_stage_2(x_map)

        x_seg_fea = self.seg_stage(x_map_1)
        #x_seg = self.seg_pred(x_seg_fea)

        x_seg_fea = self.trans_den(x_seg_fea)
        x = self.max1(x_map)
        x = self.max1(x)
        y = self.max1(x_seg_fea)
        y = self.max1(y)
        x_map = self.den_pred(torch.cat((x,y),1))
        #print("x ",x_map.size())

        x = self.max1(x_map)
        x = self.max1(x_map)        
        #x = self.output_layer(x)
        #print("c", x.size())


        return x #x_map#, x_bbx, x_seg

In [ ]:
'''
#test_funcs, dataset, set_name, network, print_output=False, thresh=0.2

model = ori()
print(model)
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params )
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params )


x = torch.randn(1, 3, 768, 1024)

# Let's print it
model(x)


'''


In [ ]:
import sys
import os

import warnings

# import from library
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import argparse
import json
import cv2
import time

"""
A dublicate of train.py 
However it does not need commandline arg
"""
def main():
    global args, best_prec1
    args = make_meow_args()


    best_prec1 = 1e6

    args.original_lr = 1e-7
    args.lr = 1e-7
    args.batch_size = 1
    args.momentum = 0.95
    args.decay = 5 * 1e-4
    args.start_epoch = 0
    args.epochs = 100
    args.steps = [-1, 1, 100, 150]
    args.scales = [1, 1, 1, 1]
    args.workers = 4
    args.seed = time.time()
    args.print_freq = 30
    args.pre = PRETRAINED_MODEL

    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    torch.cuda.manual_seed(args.seed)

    train_list, val_list = get_train_val_list(DATA_PATH)

    model = ori()

    model = model.cuda()

    criterion = nn.MSELoss(size_average=False).cuda()

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.decay)

    if args.pre:
        if os.path.isfile(args.pre):
            print("=> loading checkpoint '{}'".format(args.pre))
            checkpoint = torch.load(args.pre)
            args.start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.pre, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.pre))

    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer, epoch)

        train(train_list, model, criterion, optimizer, epoch)
        prec1 = validate(val_list, model, criterion)
        is_best = prec1 < best_prec1
        best_prec1 = min(prec1, best_prec1)
        print(' * best MAE {mae:.3f} '
              .format(mae=best_prec1))
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': args.pre,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, args.task)


def train(train_list, model, criterion, optimizer, epoch):
    losses = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    train_loader = torch.utils.data.DataLoader(
        ListDataset(train_list,
                            shuffle=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                            std=[0.229, 0.224, 0.225]),
                            ]),
                            train=True,
                            seen=model.seen,
                            batch_size=args.batch_size,
                            num_workers=args.workers),
        batch_size=args.batch_size)
    print('epoch %d, processed %d samples, lr %.10f' % (epoch, epoch * len(train_loader.dataset), args.lr))

    model.train()
    end = time.time()

    for i, (img, target) in enumerate(train_loader):
        data_time.update(time.time() - end)

        img = img.cuda()
        img = Variable(img)
        output = model(img)

        target = target.type(torch.FloatTensor).unsqueeze(0).cuda()
        target = Variable(target)
        
        #print('out ', output.size(),' tar ',target.size())
        #target.reshape([1, 1, 34, 120]) #out[1, 1, 34, 120])#tar[1, 1, 67, 120]
        loss = criterion(output, target)

        losses.update(loss.item(), img.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                .format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses))


def validate(val_list, model, criterion):
    print('begin test')
    test_loader = torch.utils.data.DataLoader(
        ListDataset(val_list,
                            shuffle=False,
                            transform=transforms.Compose([
                                transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                            std=[0.229, 0.224, 0.225]),
                            ]), train=False),
        batch_size=args.batch_size)

    model.eval()

    mae = 0

    for i, (img, target) in enumerate(test_loader):
        img = img.cuda()
        img = Variable(img)
        output = model(img)

        mae += abs(output.data.sum() - target.sum().type(torch.FloatTensor).cuda())
        #print('kkkk',format(mae))

    mae = mae / len(test_loader)
    print(' * MAE {mae:.3f} '
          .format(mae=mae))

    return mae

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""

    args.lr = args.original_lr

    for i in range(len(args.steps)):

        scale = args.scales[i] if i < len(args.scales) else 1

        if epoch >= args.steps[i]:
            args.lr = args.lr * scale
            if epoch == args.steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = args.lr

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
if __name__ == '__main__':

    main()

In [ ]:
test_image_list = create_training_image_list(TEST_DATA_PATH)

In [ ]:
best_checkpoint = torch.load("task_one_checkpoint.pth.tar")
model = ori()
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.decay)
criterion = nn.MSELoss(size_average=False).cuda()
model.load_state_dict(best_checkpoint['state_dict'])
optimizer.load_state_dict(best_checkpoint['optimizer'])
model = model.cuda()

In [ ]:
test_result = validate(test_image_list, model, criterion)

In [ ]:
best_checkpoint = torch.load("task_one_model_best.pth.tar")
model = ori()
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.decay)
criterion = nn.MSELoss(size_average=False).cuda()
model.load_state_dict(best_checkpoint['state_dict'])
optimizer.load_state_dict(best_checkpoint['optimizer'])
model = model.cuda()

In [ ]:
test_result = validate(test_image_list, model, criterion)

In [ ]:
val_list=create_training_image_list(TEST_DATA_PATH)

In [ ]:
import glob
# importing libraries
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
from matplotlib import cm as CM
import torch
from tqdm import tqdm
%matplotlib inline

from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])
model = ori()
model = model.cuda()
checkpoint = torch.load('./task_one_model_best.pth.tar')
model.load_state_dict(checkpoint['state_dict'])


In [ ]:
from matplotlib import cm as c
for i in range(len(val_list)):
    print('frame:',i)
    #print('kkkk',os.path.splitext(os.path.basename(val_list[i]))[0])
    ll=os.path.splitext(os.path.basename(val_list[i]))[0]
    img = transform(Image.open(val_list[i]).convert('RGB')).cuda()
    output = model(img.unsqueeze(0))
    #print("Predicted Count : ",int(output.detach().cpu().sum().numpy()))
    temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(int(output.detach().cpu().sum().numpy()))
    plt.imshow(temp,cmap = c.jet)
    plt.savefig('./data_results/SCNet_'+ ll + '.png')
    plt.show()
    
    temp = h5py.File('./Data/test_data/ground-truth/' + ll + '.h5', 'r')
    temp_1 = np.asarray(temp['density'])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(int(np.sum(temp_1)) + 1)
    plt.imshow(temp_1,cmap = c.jet)
    plt.savefig('./data_results/'+ ll + '_GT.png')
    print("Original Count : ",int(np.sum(temp_1)) + 1)
    